# Provenance Graph Building Scorer
ProvenanceGraphBuildingScorer.py
## Description

This script performs scoring for the Medifor Provenance Graph Building task.  The script produces output files in the form of a pipe-separated csv files, and returns an exit status of 0 if scoring was successful.  If the script aborts due to an error, the error will be written to standard out, and the script will exit with a status of 1.  The script produces two output files, 'trial_scores.csv', which reports scores at the trial (probe) level, and 'scores.csv', which aggregates metrics over trials.  The script also generates both a 'node_mapping.csv' and a 'link_mapping' file which lists individual nodes and links, respectively, for each trial and it's designation as either "Correct", "FalseAlarm", or "Missing".

The script reports the following metrics:
* SimNLO - Similarity of node link overlap
* SimNO - Similarity of node overlap
* SimLO - Similarity of link overlap
* Node recall - Recall for the nodes reported by the system

Where:
$$\begin{equation*}
Sim_{NLO}(G, G') = 2\frac{|N \cap N'| + |L \cap L'|}{|N| + |N'| + |L| + |L'|}
\end{equation*}$$

$$\begin{equation*}
Sim_{NO}(G, G') = 2\frac{|N \cap N'|}{|N| + |N'|}
\end{equation*}$$

$$\begin{equation*}
Sim_{LO}(G, G') = 2\frac{|L \cap L'|}{|L| + |L'|}
\end{equation*}$$

$$\begin{equation*}
recall = \frac{|\{relevant\} \cap \{retrieved\}|}{|\{relevant\}|}
\end{equation*}$$

The script also reports various counts, such as the number of correct nodes and links.  See the examples section for sample output.

## Command-line interface

### Required Arguments

-o, --output-dir OUTPUT_DIR
* Output directory, where you want the script to write results

-x, --index-file INDEX_FILE
* Index file (e.g. NC2017_Beta/indexes/NC2017_Dev2-provenancefiltering-index.csv)

-r, --reference-file REFERENCE_FILE
* Reference file (e.g. NC2017_Beta/reference/provenancefiltering/NC2017_Dev2-provenancefiltering-ref.csv)

-n, --node-file NODE_FILE
* Node file (e.g. NC2017_Beta/reference/provenancefiltering/NC2017_Dev2-provenancefiltering-node.csv)

-w, --world-file WORLD_FILE
* World file (e.g. NC2017_Beta/indexes/NC2017_Dev2-provenancefiltering-world.csv)

-R, --reference-dir REFERENCE_DIR
* Reference directory (e.g. NC2017_Beta/), relative paths to journals in the REFERENCE_FILE are relative to this directory

-s, --system-output-file SYSTEM_OUTPUT_FILE
* System output file (e.g. `<EXPID>`.csv)

-S, --system-dir SYSTEM_DIR
* System output directory where system output json files can be found.  Paths to system output json files in the SYSTEM_OUTPUT_FILE should be relative to this directory

### Optional Arguments

-v, --verbose
* If set, scoring progress messages and warnings are logged to standard output

-t, --skip-trial-disparity-check
* By default the script will check that the trials in INDEX_FILE correspond one-to-one to the trials in the SYSTEM_OUTPUT_FILE.  If this option is set, extraneous trials in the SYSTEM_OUTPUT_FILE are reported as warnings instead of errors.  Note that warnings are suppressed unless the -v option is set

-d, --direct
* Toggles on direct path scoring

-p, --plot-scored
* If enabled, a graphical representation of each trial's mapping will be produced, where green nodes/links represent correct responses, red represents false alarms, and gray represents misses.  These figures will be placed in `<OUTPUT_DIR>`/figures/ as `<ProvenanceProbeFileID>`.jpg.  Note that this option requires an additional Python library be installed: 'pydot'.

-H, --html-report
* Generates an HTML report file which includes tables for trial-level and aggregate scores, as well as links to the graphical representations for each trial (this option forces the -p option).  The report is saved as `<OUTPUT_DIR>`/report.html

-T, --thumbnail-cache-dir THUMBNAIL_CACHE_DIR
* If used in conjunction with the -p option, thumbnails for the graphical output are placed in THUMBNAIL_CACHE_DIR.  Thumbnails are not removed at the end of a scoring run

-c, --warn-on-system-cycle
* By default the script aborts if a cycle is detected in a system output JSON.  This flag downgrades that error to a warning.  Note that warnings are suppressed unless the -v option is set

## Example Usage

The following example uses the test case files included with the tool


In [1]:
%%bash
./ProvenanceGraphBuildingScorer.py -o "compcheckfiles/graphbuilding_example/" \
                                   -x "../../data/test_suite/provenanceScorerTests/test_case_2-provenancegraphbuilding-index.csv" \
                                   -r "../../data/test_suite/provenanceScorerTests/test_case_2-provenance-ref.csv" \
                                   -n "../../data/test_suite/provenanceScorerTests/test_case_2-provenance-node.csv" \
                                   -w "../../data/test_suite/provenanceScorerTests/test_case_2-provenancegraphbuilding-world.csv" \
                                   -R "../../data/test_suite/provenanceScorerTests/" \
                                   -s "../../data/test_suite/provenanceScorerTests/test_case_2-system_output_1_index.csv" \
                                   -S "../../data/test_suite/provenanceScorerTests/"
                                   
head compcheckfiles/graphbuilding_example/scores.csv \
     compcheckfiles/graphbuilding_example/trial_scores.csv \
     compcheckfiles/graphbuilding_example/node_mapping.csv \
     compcheckfiles/graphbuilding_example/link_mapping.csv

==> compcheckfiles/graphbuilding_example/scores.csv <==
Direct|MeanSimNLO|MeanSimNO|MeanSimLO|MeanNodeRecall
False|0.545454545455|0.789473684211|0.307692307692|0.833333333333

==> compcheckfiles/graphbuilding_example/trial_scores.csv <==
JournalName|ProvenanceProbeFileID|Direct|ProvenanceOutputFileName|NumSysNodes|NumSysLinks|NumRefNodes|NumRefLinks|NumCorrectNodes|NumMissingNodes|NumFalseAlarmNodes|NumCorrectLinks|NumMissingLinks|NumFalseAlarmLinks|SimNLO|SimNO|SimLO|NodeRecall
c257f3e674aaa597784ae6f5a402c748|9f796693249720d9d4eb8f1c3347c300|False|jsons/test_case_2-system_output_1.json|20|21|18|18|15|3|5|6|12|15|0.545454545455|0.789473684211|0.307692307692|0.833333333333

==> compcheckfiles/graphbuilding_example/node_mapping.csv <==
JournalName|ProvenanceProbeFileID|Direct|ProvenanceOutputFileName|WorldFileID|Mapping
c257f3e674aaa597784ae6f5a402c748|9f796693249720d9d4eb8f1c3347c300|False|jsons/test_case_2-system_output_1.json|26ed524291d79ad35696bbe856ba7e38|Correct
c257f3e674aaa5977

## Disclaimer

This software was developed at the National Institute of Standards
and Technology (NIST) by employees of the Federal Government in the
course of their official duties. Pursuant to Title 17 Section 105
of the United States Code, this software is not subject to copyright
protection and is in the public domain. NIST assumes no responsibility
whatsoever for use by other parties of its source code or open source
server, and makes no guarantees, expressed or implied, about its quality,
reliability, or any other characteristic.